In [1]:
# Initialize OpenAI (if Needed)
import sys

# Add the parent directory to the system path
sys.path.append('..')

from utils import BeatToStory

beatbot = BeatToStory()


## Prose Generation Coding Task

The challenge involves creating a backend system that transforms a series of bulleted "beats" into well-crafted prose that you would find in a novel. This tool is designed to aid writers in fleshing out as scene based on their outlines.

**Part 1: Beats-to-Prose**
Given a set of story beats, like this set for a sci-fi chapter:

1. Begin the chapter with Jack and Xander continuing their excavation on the lunar surface,
creating a sense of tension and anticipation.
2. Describe the barren landscape of the moon, emphasizing its desolation and the isolation
felt by Jack and Xander.
3. Use vivid and descriptive language to portray the moment when Jack and Xander uncover
an alien artifact, highlighting its mysterious and otherworldly appearance.
4. Convey Jack's excitement and trepidation as he realizes the significance of the discovery
and the potential implications it holds for humanity.
5. Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the
urgency of the situation.
6. Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship
and the gravity of the situation.
7. Highlight the importance of the discovery and the potential consequences it could have on
Earth and the lunar mining project.
8. Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a
sense of anticipation and uncertainty.
9. Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges
and dangers that Jack and Xander face.
10. Use sensory details to immerse the reader in the lunar environment, such as the crunch of
lunar soil beneath their boots and the cold, airless atmosphere.
11. Portray Jack's determination and resolve to protect the discovery, even in the face of
potential conflict with rival miners.
12. Hint at the potential dangers and obstacles that Jack and Xander may encounter as they
navigate through a world of corporate greed and betrayal.
13. Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and
anticipation for future chapters.
14. End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to
continue reading and discover what happens next.

Use these beats to produce ~1500 words of prose that matches these beats - in other words, if
a human were to summarize the prose, they would produce something along the lines of these
beats. It's okay to hallucinate details if they are missing from the beats in order to create the
narrative. Target approximately 100-150 words of output per beat. You are allowed to use any LLM or NLP techniques, including open source and closed source services like GPT or Claude.




In [2]:
# Test Input
TEST_BEATS_INPUT = [
    "Begin the chapter with Jack and Xander continuing their excavation on the lunar surface, creating a sense of tension and anticipation.",
    "Describe the barren landscape of the moon, emphasizing its desolation and the isolation felt by Jack and Xander.",
    "Use vivid and descriptive language to portray the moment when Jack and Xander uncover an alien artifact, highlighting its mysterious and otherworldly appearance.",
    "Convey Jack's excitement and trepidation as he realizes the significance of the discovery and the potential implications it holds for humanity.",
    "Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the urgency of the situation.",
    "Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship and the gravity of the situation.",
    "Highlight the importance of the discovery and the potential consequences it could have on Earth and the lunar mining project.",
    "Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a sense of anticipation and uncertainty.",
    "Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges and dangers that Jack and Xander face.",
    "Use sensory details to immerse the reader in the lunar environment, such as the crunch of lunar soil beneath their boots and the cold, airless atmosphere.",
    "Portray Jack's determination and resolve to protect the discovery, even in the face of potential conflict with rival miners.",
    "Hint at the potential dangers and obstacles that Jack and Xander may encounter as they navigate through a world of corporate greed and betrayal.",
    "Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and anticipation for future chapters.",
    "End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to continue reading and discover what happens next."
]

## Agentic with context

In [3]:
beatbot.max_words_per_beat

150

In [4]:
beatbot.beats = TEST_BEATS_INPUT
beatbot

BeatToStory(min_words_per_beat=100, max_words_per_beat=150, max_attempts_per_beat=10, story='', edited_story='', beats=['Begin the chapter with Jack and Xander continuing their excavation on the lunar surface, creating a sense of tension and anticipation.', 'Describe the barren landscape of the moon, emphasizing its desolation and the isolation felt by Jack and Xander.', 'Use vivid and descriptive language to portray the moment when Jack and Xander uncover an alien artifact, highlighting its mysterious and otherworldly appearance.', "Convey Jack's excitement and trepidation as he realizes the significance of the discovery and the potential implications it holds for humanity.", 'Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the urgency of the situation.', 'Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship and the gravity of the situation.', 'Highlight the importance of the discovery and the potential consequen

In [5]:
beatbot.generate_story(verbose= True)

    crafting context on beat 0
    crafting context on beat 1
    crafting context on beat 2
    crafting context on beat 3
    crafting context on beat 4
    crafting context on beat 5
    crafting context on beat 6
    crafting context on beat 7
    crafting context on beat 8
    crafting context on beat 9
    crafting context on beat 10
    crafting context on beat 11
    crafting context on beat 12
90
100
150
        beat 1 | attempt: 0 | Inconsistency detected; regenerating passage...
103
100
150
124
100
150
104
100
150
114
100
150
        beat 5 | attempt: 0 | Inconsistency detected; regenerating passage...
107
100
150
115
100
150
110
100
150
97
100
150
107
100
150
115
100
150
119
100
150
123
100
150


'Jack and Xander\'s tools clinked against the lunar surface, the tension palpable as they dug deeper, their breath fogging up their helmets. The barren landscape stretched endlessly around them, a stark reminder of their isolation. Each scoop of moon dust unveiled mysteries and dangers, fueling their anticipation. The desolation of the moon mirrored the emptiness in their hearts, a void only the secrets hidden beneath the surface could fill. As they worked, the silence of space enveloped them, amplifying the weight of their mission and the unknown that lay ahead.\nAs Jack and Xander delved deeper into the lunar surface, their tools unearthed an alien artifact unlike anything they had ever seen. It shimmered with an ethereal glow, casting iridescent hues across the desolate landscape. The artifact pulsed softly, as if resonating with a hidden energy, captivating the astronauts with its mysterious and otherworldly appearance. Its intricate patterns seemed to shift and dance, defying the 

### **Part 2: Incorporating Story Metadata**

Now we want to extend the functionality of Beats-to-Prose to accept a few new parameters:
- a list of characters (name and details about them)
- setting
- genre
- style of prose and have that alter the generated prose.

Describe in detail your approach to this. Some things to consider:

1. What models would you use? Why?
2. If you need to procure data, how would go about doing that? How would you prepare the data?
3. What is your approach to producing prose that matches the parameters?

How would you evaluate whether the generations are good quality and fit the parameters specified?
Consider cost and latency considerations

In [5]:
# Write your solution for alterations here. Feel free to make methods or classes to do so


Input looks like:

```
{
}
```


## **Part 3: Describe how you might turn this code into an API**

Imagine you have a working protoype and would like the application team to start using your solution. You can assume you are using the FastAPI framework or something similar. You can use psuedocode for this exercise.

1. What would a hander look like, and what parameters would it receive?

In [ ]:
# Write your solution for the handler here You don't have to run this cell
from fastapi import FastAPI
from fastapi.routing import APIRoute

app = FastAPI()

@app.post("")
async def beat_to_prose_generation():
    return []
